In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature
from azure.ai.documentintelligence.models._models import AnalyzeResult

import pandas as pd
import numpy as np
import functools
import threading
import json
import io
import re
import os
import uuid
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

True

In [18]:
key = os.environ.get('OCR_KEY')
endpoint = os.environ.get('OCR_ENDPOINT')
credential = AzureKeyCredential(key)
document_intelligence_client = DocumentIntelligenceClient(endpoint, credential, api_version='2023-10-31-preview')
doc_path ='data/handwritten-form.jpg'

with open(doc_path, "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-read", 
        analyze_request=f, 
        content_type="application/octet-stream", 
    )
r_cloud = poller.result()

In [16]:
import requests
import json

service_url = 'http://localhost:5000/'

# Define the URL and file path
url = f"{service_url}formrecognizer/documentModels/prebuilt-read:syncAnalyze?api-version=2022-08-31"
file_path = "data/handwritten-form.jpg"

# Define headers
headers = {
    'accept': '*/*',
    'Content-Type': 'application/octet-stream',
}

# Open the file and send the POST request
with open(file_path, 'rb') as file:
    response = requests.post(url, headers=headers, data=file)

r_local = json.loads(response.text)
r_local = r_local['analyzeResult']

In [19]:
print(f'cloud keys: {r_cloud.keys()}')
print(f'local keys: {r_local.keys()}')

cloud keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'contentFormat'])
local keys: dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'paragraphs', 'styles', 'languages'])


In [31]:
print(f'cloud keys: {r_cloud["pages"][0].keys()}')
print(f'local keys: {r_local["pages"][0].keys()}')

print(f'cloud keys: {r_cloud["styles"]}')
print(f'local keys: {r_local["styles"]}')

cloud keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans'])
local keys: dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans', 'kind'])
cloud keys: [{'confidence': 1, 'spans': [{'offset': 34, 'length': 6}, {'offset': 56, 'length': 20}, {'offset': 303, 'length': 32}, {'offset': 339, 'length': 2}, {'offset': 346, 'length': 3}, {'offset': 354, 'length': 3}, {'offset': 374, 'length': 5}, {'offset': 387, 'length': 6}, {'offset': 399, 'length': 4}, {'offset': 410, 'length': 5}, {'offset': 423, 'length': 6}, {'offset': 433, 'length': 2}, {'offset': 441, 'length': 4}, {'offset': 452, 'length': 5}, {'offset': 465, 'length': 6}, {'offset': 475, 'length': 2}, {'offset': 482, 'length': 3}, {'offset': 492, 'length': 5}, {'offset': 505, 'length': 9}, {'offset': 519, 'length': 3}, {'offset': 543, 'length': 6}, {'offset': 553, 'length': 2}, {'offset': 560, 'length': 3}, {'offset': 569, 'length': 4}, {'offset': 580, 'length': 5},

In [21]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0].keys()}')
print(f'local keys: {r_local["pages"][0]["words"][0].keys()}')

cloud keys: dict_keys(['content', 'polygon', 'confidence', 'span'])
local keys: dict_keys(['content', 'polygon', 'confidence', 'span'])


In [27]:
print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["polygon"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["polygon"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["content"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["content"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["confidence"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["confidence"]}')

print(f'cloud keys: {r_cloud["pages"][0]["words"][0]["span"]}')
print(f'local keys: {r_local["pages"][0]["words"][0]["span"]}')

cloud keys: [187, 15, 313, 15, 313, 34, 187, 33]
local keys: [186, 13, 315, 14, 314, 37, 186, 35]
cloud keys: HANDWRITING
local keys: HANDWRITING
cloud keys: 0.993
local keys: 0.991
cloud keys: {'offset': 0, 'length': 11}
local keys: {'offset': 0, 'length': 11}
